In [97]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [98]:
# Extract
file = "SARS_2003.csv"
sars = pd.read_csv(file)
sars.head()

,Date,Country,Cumulative number of case(s),Number of deaths,Number recovered
0,3/17/2003,Germany,1,0,0
1,3/17/2003,Canada,8,2,0
2,3/17/2003,Singapore,20,0,0
3,3/17/2003,"Hong Kong SAR, China",95,1,0
4,3/17/2003,Switzerland,2,0,0


In [99]:
sars = sars[['Country', 'Cumulative number of case(s)', 'Number of deaths']].copy()
# sars.head()

In [100]:
# Rename the column headers
sars_clean = sars.rename(columns={"Country": "country",
                                 "Cumulative number of case(s)": "number_of_cases_sars",
                                 "Number of deaths": "number_of_deaths_sars"})
sars_clean.head()

,country,number_of_cases_sars,number_of_deaths_sars
0,Germany,1,0
1,Canada,8,2
2,Singapore,20,0
3,"Hong Kong SAR, China",95,1
4,Switzerland,2,0


In [101]:
# Extract
file1 = "TB_2007.csv"
tb = pd.read_csv(file1)
tb = tb[['Country', 'Number of prevalent tuberculosis cases', 'Deaths due to tuberculosis among HIV-negative people (per 100 000 population)']].copy()

In [102]:
tb['Number of prevalent tuberculosis cases'] = tb['Number of prevalent tuberculosis cases'].apply(lambda x:x.replace(' ','')).astype(float)
# tb.head()

In [109]:
tb_clean = tb.rename(columns={"Country": "country",
                                 "Number of prevalent tuberculosis cases": "number_of_cases_tb",
                                 "Deaths due to tuberculosis among HIV-negative people (per 100 000 population)": "number_of_deaths_tb"})
tb_clean.head()

,country,number_of_cases_tb,number_of_deaths_tb
0,Afghanistan,110000.0,44.0
1,Afghanistan,100000.0,42.0
2,Afghanistan,98000.0,40.0
3,Afghanistan,94000.0,40.0
4,Afghanistan,91000.0,39.0


In [104]:
# Extract
file2 = "HN_2009.csv"
hn = pd.read_csv(file2, encoding= 'latin1')
# hn.head()

In [105]:
# Transform
hn = hn[['Country', 'Cases','Deaths']]

In [106]:
# Transform
hn_clean = hn.rename(columns={"Country": "country",
                                 "Cases": "number_of_cases_h1n1",
                                 "Deaths": "number_of_deaths_h1n1"})
hn_clean.head()

,country,number_of_cases_h1n1,number_of_deaths_h1n1
0,Algeria,5,0.0
1,Antigua and Barbuda,2,0.0
2,Argentina,2485,60.0
3,Australia,5298,10.0
4,Austria,19,0.0


In [110]:
# Merge the dataframes to create one large df
# Transform
df_1 = pd.merge(sars_clean,tb_clean, on='country')
# df_1.head()

df_new = pd.merge(df_1,hn_clean, on='country')
# df_new.set_index('country', inplace=True)

In [123]:
#df_f = df_new.sort_values('country', ascending=True)
#df_f.reset_index(inplace=True)
df_f.drop(columns=['index'], axis=1, inplace=True)
df_f.head()

,country,number_of_cases_sars,number_of_deaths_sars,number_of_cases_tb,number_of_deaths_tb,number_of_cases_h1n1,number_of_deaths_h1n1
0,Australia,3,0,1900.0,0.18,4568,9.0
1,Australia,1,0,1700.0,0.20,1051,0.0
2,Australia,1,0,1700.0,0.20,1224,0.0
3,Australia,1,0,1700.0,0.20,1307,0.0
4,Australia,1,0,1700.0,0.20,1307,0.0


In [112]:
# Load

connection_string = "postgres:postgres@localhost:5433/pandemics"
engine = create_engine(f'postgresql://{connection_string}')

In [113]:
engine.table_names()

['pandemic_data']

In [124]:
df_f.to_sql(name='pandemic_data', con=engine, if_exists='append', index=False)

In [125]:
pd.read_sql_query('select * from pandemic_data', con=engine).head()

,country,number_of_cases_sars,number_of_deaths_sars,number_of_cases_tb,number_of_deaths_tb,number_of_cases_h1n1,number_of_deaths_h1n1
0,Australia,3,0,1900.0,0.18,4568,9.0
1,Australia,1,0,1700.0,0.2,1051,0.0
2,Australia,1,0,1700.0,0.2,1224,0.0
3,Australia,1,0,1700.0,0.2,1307,0.0
4,Australia,1,0,1700.0,0.2,1307,0.0
